In [12]:
import feedparser
import pandas as pd
import boto3
import json
from botocore.exceptions import ClientError
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np

In [2]:
RSS_list = [
    # --- グローバル・メガテック公式（最先端技術・基盤モデルの動向） ---
    "https://openai.com/news/rss.xml",           # OpenAI (業界のゲームチェンジャー、最新モデル発表)
    "https://cloudblog.withgoogle.com/rss/",     # Google Cloud Blog (Gemini/Vertex AI関連)
    "https://aws.amazon.com/jp/blogs/aws/feed/", # AWS News Blog (Bedrock/SageMaker関連)
    "https://developers-jp.googleblog.com/atom.xml", # Google Developers Japan (国内向け開発情報)

    # --- 国内メガベンチャー・テックブログ（実践的な実装・導入事例） ---
    "https://engineering.mercari.com/blog/feed.xml", # Mercari (大規模プロダクトへのAI実装)
    "https://developers.cyberagent.co.jp/blog/feed/", # CyberAgent (広告AI、LLM独自開発)
    "https://www.ai-shift.co.jp/techblog/feed",  # AI Shift (カスタマーサポートAI、RAG実践)
    "https://recruit-tech.co.jp/blog/feed/",     # Recruit (大規模データ活用・マッチングAI)
    "https://blog.brainpad.co.jp/rss",           # ブレインパッド (DXコンサル・データサイエンス)

    # --- 開発コミュニティ・技術メディア（現場のナレッジ・最新トレンド） ---
    "https://zenn.dev/topics/llm/feed",          # Zenn LLM (国内エンジニアの最速Tips・試行錯誤)
    "https://rss.itmedia.co.jp/rss/2.0/aiplus.xml", # ITmedia AI+ (最新ニュース・国内動向)
    "https://atmarkit.itmedia.co.jp/rss/rss.xml", # ＠IT (深掘り技術解説・マネジメント向け)
#    "https://xtech.nikkei.com/rss/xtech-it.rdf", # 日経クロステック (ビジネス視点のIT・AIニュース) #published_parsedがないかも
    "https://medium.com/feed/@kyakuno",          # Medium @kyakuno (技術選定・アーキテクチャ考察)

    # --- 論文・リサーチ（SOTA・学術的ブレイクスルー） ---
    "https://rsshub.app/huggingface/daily-papers", # Hugging Face Daily Papers (最重要論文)
    "https://research.preferred.jp/feed/",       # PFN Research (日本最高峰のAI研究)
    "https://feeds.feedburner.com/blogspot/gJZg", # Google Research Blog (グローバルな研究成果)
    "https://pytorch.org/feed.xml",              # PyTorch Official (深層学習フレームワークの進化)
#    "https://huggingface.co/blog/feed.xml",      # Hugging Face Blog (オープンソースモデルのハブ) #summaryがないかも

    # --- 海外ニュースレター・分析（戦略・マクロ・倫理） ---
    "https://www.deeplearning.ai/the-batch/rss", # The Batch (Andrew Ng氏監修の質の高い週刊分析)
    "https://www.technologyreview.jp/feed/",      # MIT Technology Review (技術の社会・倫理的インパクト)
]

In [9]:
id_1 = 1
today = datetime.now().strftime("%Y%m%d")
category = "AI_news"

news_df = pd.DataFrame()

for url in RSS_list:
    feed = feedparser.parse(url)
    id_2 = 0
    id_3 = 1
    for entry in feed.entries[0:1]:
        print(f"ID: {today}{id_1}{id_2}{id_3}")
        print(f"カテゴリー: {category}")
        print(f"タイトル: {entry.title}")
        print(f"リンク: {entry.link}")
        struct_time = entry.published_parsed
        dt = datetime(*struct_time[:6])
        print(f"公開日時 (datetime): {dt}")
        print(f"概要: {entry.summary}")

        addRow = [f"{today}{id_1}{id_2}{id_3}", category,  entry.title, entry.link, dt.isoformat(), entry.summary]
        addRow = pd.DataFrame([addRow], columns=["id", "category", "title", "link", "published_datetime", "summary"])
        news_df = pd.concat([news_df, addRow], ignore_index=True)
        print("-" * 30)

        if id_3 == 9:
            id_2 += 1
        
        if id_3 == 9:
            id_3 = 0
        else:
            id_3 += 1

    id_1 += 1

ID: 20260201101
タイトル: Inside OpenAI’s in-house data agent
リンク: https://openai.com/index/inside-our-in-house-data-agent
公開日時 (datetime): 2026-01-29 10:00:00
概要: How OpenAI built an in-house AI data agent that uses GPT-5, Codex, and memory to reason over massive datasets and deliver reliable insights in minutes.
------------------------------
ID: 20260201201
タイトル: What Google Cloud announced in AI this month
リンク: https://cloud.google.com/blog/products/ai-machine-learning/what-google-cloud-announced-in-ai-this-month/
公開日時 (datetime): 2026-01-30 17:00:00
概要: <div class="block-paragraph"><p><b><i>Editor’s note</i></b><i>: Want to keep up with the latest from Google Cloud? Check back here for a monthly recap of our latest updates, announcements, resources, events, learning opportunities, and more.</i></p><hr /><p></p></div>
<div class="block-paragraph_advanced"><p><span style="vertical-align: baseline;">We used to have to learn the language of computers. In 2026, they’re learning ours.</span

In [10]:
# HTMLタグを除去する関数
def clean_html(html, strip=False):
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(strip=strip)
    return text
news_df["summary"] = news_df["summary"].apply(lambda x: clean_html(x,strip=True))
news_df

,id,category,title,link,published_datetime,summary
0,20260201101,AI_news,Inside OpenAI’s in-house data agent,https://openai.com/index/inside-our-in-house-d...,2026-01-29T10:00:00,How OpenAI built an in-house AI data agent tha...
1,20260201201,AI_news,What Google Cloud announced in AI this month,https://cloud.google.com/blog/products/ai-mach...,2026-01-30T17:00:00,Editor’s note: Want to keep up with the latest...
2,20260201301,AI_news,"AWS Weekly Roundup: Amazon EC2 G7e instances, ...",https://aws.amazon.com/blogs/aws/aws-weekly-ro...,2026-01-26T16:25:46,"Hey! It’s my first post for 2026, and I’m writ..."
3,20260201401,AI_news,【Next Tokyo】120 以上のアーカイブ セッションを本日公開！,https://developers-jp.googleblog.com/2025/08/n...,2025-08-21T02:00:00,すでに公開していた基調講演（DAY 1|DAY 2）に加え、120 以上のブレイクアウト セ...
4,20260201501,AI_news,TTLを利用したTiDBの効率的なインスタンス活用,https://engineering.mercari.com/blog/entry/202...,2026-01-06T07:00:46,MySQLに高い互換性を持つデータベースのTiDBには、古いデータを自動的に削除するTTL(...
5,20260201601,AI_news,ゲーム業界駆け出しエンジニアがCodexを使いUnityゲームをソフトローンチした話,https://developers.cyberagent.co.jp/blog/archi...,2026-01-28T06:18:20,はじめに こんにちは、株式会社GOODROID（ハイパーカジュアル系スマホゲーム開発）の洞秀...
6,20260201701,AI_news,OpenEvals × Langfuseで始めるAIエージェントのマルチターン評価,https://www.ai-shift.co.jp/techblog/6705,2026-01-26T09:13:59,こんにちは、AIチームの長澤 (@sp_1999N) です。 Claude Codeなどを代...
7,20260201901,AI_news,技術への探求心を、社外と繋がるきっかけへ。エンジニア主催の勉強会「DEU Tech Driv...,https://blog.brainpad.co.jp/entry/2026/01/29/1...,2026-01-29T05:00:00,本記事では、ブレインパッドのデータエンジニアリングユニット（以下、DEユニット）が主催する、...
8,202602011001,AI_news,RedSageでサイバーセキュリティ支援を強化し効率的な対応を実現した経験,https://zenn.dev/yunisuta/articles/20260131_18...,2026-01-31T23:08:41,RedSageでサイバーセキュリティ支援を強化し効率的な対応を実現した経験 🛡️\n\n 1...
9,202602011101,AI_news,AI同士で相談させて“最強の布陣”を組む新技術 日立製作所が開発,https://www.itmedia.co.jp/enterprise/articles/...,2026-01-30T23:00:00,日立製作所が発表した新技術によって、AI同士の相性を特定し、最強のチームを自動編成することが...


In [ ]:
news_json = news_df.to_dict(orient='records')
table_name = 'AI_news_daily_db_test2'
region_name = 'ap-northeast-1'

# dynamodbの設定
dynamodb = boto3.resource('dynamodb', region_name=region_name)
table = boto3.resource('dynamodb').Table(table_name)

# 一括書き込み
with table.batch_writer() as batch:
    for item in news_json:
        item_not_has_nan = {key: item[key] for key in item if item[key] is not np.nan}
        batch.put_item(Item=item_not_has_nan)

[{'id': '20260201101',
  'category': 'AI_news',
  'title': 'Inside OpenAI’s in-house data agent',
  'link': 'https://openai.com/index/inside-our-in-house-data-agent',
  'published_datetime': '2026-01-29T10:00:00',
  'summary': 'How OpenAI built an in-house AI data agent that uses GPT-5, Codex, and memory to reason over massive datasets and deliver reliable insights in minutes.'},
 {'id': '20260201201',
  'category': 'AI_news',
  'title': 'What Google Cloud announced in AI this month',
  'link': 'https://cloud.google.com/blog/products/ai-machine-learning/what-google-cloud-announced-in-ai-this-month/',
  'published_datetime': '2026-01-30T17:00:00',
  'summary': 'Editor’s note: Want to keep up with the latest from Google Cloud? Check back here for a monthly recap of our latest updates, announcements, resources, events, learning opportunities, and more.We used to have to learn the language of computers. In 2026, they’re learning ours.We kicked off the year by exploring the future of agenti

In [ ]:
def load_api_key():
    load_dotenv(dotenv_path="./.env", override=True)
    os.environ["AWS_BEARER_TOKEN_BEDROCK"] = os.getenv("Bedrock_API_Key")

def create_response(prompt_text,df_json):
    client = boto3.client(
        "bedrock-runtime",
        region_name="us-east-1"
    )

    prompt = PromptTemplate(
        input_variables=["tweet_data"],
        template = prompt_text)
    prompt = prompt.format(tweet_data=df_json)

    body = json.dumps({
        "messages":[
            {"role":"user","content":prompt}
        ],
        "max_tokens":1024,
        "temperature":0.5,
        "anthropic_version":"bedrock-2023-05-31"
        
    })

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

    response = client.invoke_model(
        modelId=model_id,
        body=body
    )

    return response

load_api_key()

f = open("prompt.txt","r",encoding="utf-8")
prompt_text = f.read()
response = create_response(prompt_text,df_json)

response_body = json.loads(response.get("body").read())
answer = response_body["content"][0]["text"]
print(answer)